In [1]:
import os
import pandas as pd
import swisseph as swe
from swe_const import *
from vedicastro.utils import *
from vedicastro import VedicAstro as va

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
se_version_major, se_version_minor, se_version_patch = swe.version.split('.')
print(se_version_major,'.',se_version_minor,'.',se_version_patch)

2 . 10 . 03


In [4]:
# SE_EPH_PATH = os.getenv('SE_EPHE_PATH')
# print("SWE_EPH_FILE_PATH:",SE_EPH_PATH)
# swe.set_ephe_path(SE_EPH_PATH)

In [5]:
def decimal_to_dms(decimal_deg: float):
    deg, mins, sec, fractional_sec, sign_idx = swe.split_deg(decimal_deg, roundflag = swe.SPLIT_DEG_ZODIACAL )
    sec += fractional_sec  # Add fractional seconds to seconds
    return RASHIS[sign_idx], f"{deg:02d}:{mins:02d}:{int(sec):02d}"

def get_rl_nl_sl_data(deg : float):
    """
    Returns the  Rashi (Sign) Lord, Nakshatra, Nakshatra Pada, Nakshatra Lord, Sub Lord and Sub Sub Lord 
    corresponding to the given degree.
    """
    duration = [7, 20, 6, 10, 7, 18, 16, 19, 17]

    lords = ["Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"]

    star_lords = lords * 3 ## lords for the 27 Nakshatras

    ## Compute Sign lords
    sign_deg = deg % 360  # Normalize degree to [0, 360)
    sign_index = int(sign_deg // 30)  # Each zodiac sign is 30 degrees
    
    # Compute Nakshatra details
    nakshatra_deg = sign_deg % 13.332  # Each nakshatra is 13.332 degrees
    nakshatra_index = int(sign_deg // 13.332)  # Find the nakshatra index
    pada = int((nakshatra_deg % 13.332) // 3.325) + 1  # Each pada is 3.325 degrees

    # Ensure nakshatra_index is within bounds
    nakshatra_index = nakshatra_index % len(NAKSHATRAS)        

    # Compute SubLords
    deg = deg - 120 * int(deg / 120)
    degcum = 0
    i = 0

    while i < 9:
        deg_nl = 360 / 27
        j = i
        while True:
            deg_sl = deg_nl * duration[j] / 120
            k = j
            while True:
                deg_ss = deg_sl * duration[k] / 120
                degcum += deg_ss
                if degcum >= deg:
                    return {"Nakshatra": NAKSHATRAS[nakshatra_index] + "-" + str(pada), 
                            "RasiLord": SIGN_LORDS[sign_index], "NakshatraLord": star_lords[nakshatra_index],
                            "SubLord": lords[j], "SubSubLord": lords[k] }
                k = (k + 1) % 9
                if k == j:
                    break
            j = (j + 1) % 9
            if j == i:
                break
        i += 1  

# Display pd.DataFrames - side by side with titles using HTML display with formatted string
def display_side_by_side(df1_html: str, df2_html: str, localized_time: str, latitude: float, longitude: float, 
                         ayan: str, heading1: str = "Swisseph - House Results", heading2: str = "Flatlib - House Results"):
    html = f"""
    <b>Housewise Results for :</b> {localized_time} at <b>Lat:</b>{latitude} | <b>Long:</b> {longitude} | <b>Ayanamsha:</b> `{ayan}`
    <div style='display:flex; justify-content:space-between;'>
        <div style='width: 45%;'>
            <h2>{heading1}</h2>
            {df1_html}
        </div>
        <div style='width: 55%;'>
            <h2>{heading2}</h2>
            {df2_html}
        </div>
    </div>
    """
    display(HTML(html))        

### Common `SideReal` Ayanamsas for Vedic Astrology
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/krishnamurti_ayanamshas.html
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/spica_citra_tradition_and_lahiri_ayanamsha.html


In [6]:
SWE_AYANAMSHAS = { "Krishnamurti" : swe.SIDM_KRISHNAMURTI, "Krishnamurti_Senthilathiban": swe.SIDM_KRISHNAMURTI_VP291,
                   "Lahiri": swe.SIDM_LAHIRI, "Lahiri_1940": swe.SIDM_LAHIRI_1940, "Lahiri_ICRC" : swe.SIDM_LAHIRI_ICRC,
                   "Lahiri_VP285" : swe.SIDM_LAHIRI_VP285, 
                }

In [7]:
## Case 1
# year = 2024
# month = 8
# day = 14
# hour = 22
# minute = 31
# secs = 31
# latitude, longitude, utc = 28.6334, 77.2834, "+5:30" ## Delhi

In [8]:
## Case 2
year = 2000
month = 12
day = 30
hour = 22
minute = 31
secs = 59
latitude, longitude, utc_offset = 28.6334, 77.2834, "+5:30" ## Delhi

In [9]:
from pytz import timezone
zone = "Asia/Calcutta"
### TIMEZONE INFORMATION
tz = timezone(zone)
localized_time = tz.localize(datetime(year, month, day, hour, minute, secs))

### Compute House Positions using native `pyswisseph` library
- Across multiple `ayanamshas` and ephemeris files



In [10]:
utc_float =  utc_offset_str_to_float(utc_offset)

utc = swe.utc_time_zone(year, month, day, hour = hour, minutes = minute, seconds = secs, offset = utc_float)
_ , jd_start = swe.utc_to_jd(*utc) ## Unpacks utc tuple
current_time = jd_start

In [11]:
# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined
multi_eph_data = {}
swe.set_ephe_path('/Users/dilip.rajkumar/Documents/jpl_eph_data')
for flg_type in ["swe_eph","jpl_eph_de406e","jpl_eph_441"]:
    # Flag Setting
    if flg_type == "swe_eph":
        flg = swe.FLG_SWIEPH | swe.FLG_SIDEREAL | swe.FLG_SPEED
    else:
        flg =  swe.FLG_JPLEPH | swe.FLG_SIDEREAL | swe.FLG_SPEED
        if flg_type == "jpl_eph_de406e":
            swe.set_jpl_file("de406e.eph")
        elif flg_type == "jpl_eph_441":
            swe.set_jpl_file("de441.eph")

    multi_eph_data[flg_type] = {}
    for ayan in list(SWE_AYANAMSHAS.keys()):
        print(f"\nCalculating House Position Data for {flg_type} and {ayan} Ayanamsha")
        results = []  # List to store the results
        swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
        cusps, asmc = swe.houses_ex(current_time, latitude, longitude, b'P', flags = swe.FLG_SIDEREAL | swe.FLG_JPLEPH )
        
        ## Compute data for all cusps
        for idx, cusp in enumerate(cusps, start=1):
            sign, sign_dms = decimal_to_dms(cusp)
            res = {'Ayanamsa': ayan, "HouseID" : idx,'Sign': sign, 'DecLong': cusp,'SignDMS': sign_dms}
            res.update(get_rl_nl_sl_data(deg = cusp))
            results.append(res)

        # Store the results for the current ayan under the current flg_type        
        multi_eph_data[flg_type][ayan] = results


Calculating House Position Data for swe_eph and Krishnamurti Ayanamsha

Calculating House Position Data for swe_eph and Krishnamurti_Senthilathiban Ayanamsha

Calculating House Position Data for swe_eph and Lahiri Ayanamsha

Calculating House Position Data for swe_eph and Lahiri_1940 Ayanamsha

Calculating House Position Data for swe_eph and Lahiri_ICRC Ayanamsha

Calculating House Position Data for swe_eph and Lahiri_VP285 Ayanamsha

Calculating House Position Data for jpl_eph_de406e and Krishnamurti Ayanamsha

Calculating House Position Data for jpl_eph_de406e and Krishnamurti_Senthilathiban Ayanamsha

Calculating House Position Data for jpl_eph_de406e and Lahiri Ayanamsha

Calculating House Position Data for jpl_eph_de406e and Lahiri_1940 Ayanamsha

Calculating House Position Data for jpl_eph_de406e and Lahiri_ICRC Ayanamsha

Calculating House Position Data for jpl_eph_de406e and Lahiri_VP285 Ayanamsha

Calculating House Position Data for jpl_eph_441 and Krishnamurti Ayanamsha

Cal

In [12]:
def generate_eph_data_comparisons(ayanamsha_selected: str):
    eph_data_comparisons = {
        "Planets": list(PLANETS.keys()),
        "Ayanamsha": [ayanamsha_selected] * len(list(PLANETS.keys())),
        "Swiss_sepl18.eph_semo18.eph": pd.DataFrame(multi_eph_data["swe_eph"][ayanamsha_selected]).DecLong.tolist(),
        "JPL_de441.eph": pd.DataFrame(multi_eph_data["jpl_eph_441"][ayanamsha_selected]).DecLong.tolist(),
        "JPL_de406e.eph": pd.DataFrame(multi_eph_data["jpl_eph_de406e"][ayanamsha_selected]).DecLong.tolist()
    }
    return eph_data_comparisons

print(f"EPH-wise House Positions")
df_temp_ayan1 = pd.DataFrame(generate_eph_data_comparisons(ayanamsha_selected="Krishnamurti_Senthilathiban"))
df_temp_ayan2 = pd.DataFrame(generate_eph_data_comparisons(ayanamsha_selected="Krishnamurti"))
display_side_by_side(df_temp_ayan1.to_html(), df_temp_ayan2.to_html(), localized_time, latitude, longitude, 
                     ayan = "Krishnamurti & Krishnamurti-Senthilathiban", heading1="", heading2="")

EPH-wise House Positions


,Planets,Ayanamsha,Swiss_sepl18.eph_semo18.eph,JPL_de441.eph,JPL_de406e.eph
0,Sun,Krishnamurti_Senthilathiban,140.570036,140.570036,140.570036
1,Moon,Krishnamurti_Senthilathiban,167.696585,167.696585,167.696585
2,Mercury,Krishnamurti_Senthilathiban,197.988661,197.988661,197.988661
3,Venus,Krishnamurti_Senthilathiban,229.813822,229.813822,229.813822
4,Mars,Krishnamurti_Senthilathiban,261.497059,261.497059,261.497059
5,Jupiter,Krishnamurti_Senthilathiban,292.012698,292.012698,292.012698
6,Saturn,Krishnamurti_Senthilathiban,320.570036,320.570036,320.570036
7,Uranus,Krishnamurti_Senthilathiban,347.696585,347.696585,347.696585
8,Neptune,Krishnamurti_Senthilathiban,17.988661,17.988661,17.988661
9,Pluto,Krishnamurti_Senthilathiban,49.813822,49.813822,49.813822


**Why are the different .eph or .se1 files not resulting in any change in the house long computation for the same ayanamsha?** <br>
The lack of variation in house longitudes across different ephemeris files (Swiss_sepl18_semo18.se1, JPL_de441.eph, and JPL_de406e.eph) in your results can be attributed to several factors:
1. **Accuracy of Modern Ephemerides**: Both the Swiss Ephemeris and the JPL ephemerides (such as DE441 and DE406e) are highly accurate for contemporary dates. These ephemerides are the result of extensive observations and calculations, ensuring that their predictions for planetary positions are very close to each other, especially for dates within the last few centuries and going forward.
2. **Nature of House Calculations**: The calculation of house cusps in astrology, especially when using the `Placidus` system or other similar systems, is more directly influenced by the geographic location (latitude and longitude), time, and the chosen ayanamsa rather than the minute differences between ephemeris files. Since the primary input for house cusp calculation is the local sidereal time (or equivalent), which is derived from the given date, time, and location, the ephemeris file's role is somewhat indirect, primarily affecting the calculation of the ayanamsa if sidereal zodiac is used.
3. **Sidereal Time and Ayanamsa Consistency**: For the calculation of house positions, the critical factor is the local sidereal time, which is calculated from the given date and time. The ayanamsa (if using sidereal zodiac) adjusts the tropical positions to sidereal, but since you're using the same ayanamsa across different ephemeris files, the base reference point remains consistent. The ephemeris files are used more directly for calculating planetary positions rather than house cusps.
4. **High Precision of Ephemeris Data**: The differences between ephemeris files like DE441 and DE406e are in the precision and span of the data. DE441 is a more recent and comprehensive ephemeris compared to DE406e. However, for practical purposes, especially in astrology where the exact degree of precision is less critical than in, say, spacecraft navigation, these differences are negligible.
5. **Implementation Details**: The Swiss Ephemeris library abstracts away the differences between using its internal ephemeris versus an external JPL ephemeris for many calculations. When calculating house positions, the differences in planetary positions between ephemeris files are so minor that they don't significantly impact the outcome.<br><br>
In summary, the lack of variation in house longitudes across different ephemeris files is due to the high accuracy and consistency of modern ephemerides for the purposes of astrological calculations, along with the nature of house calculation methods which are less sensitive to the minor differences between these ephemeris files.

In [13]:
## Concatenate all ayanamshas for the Common SwissEPH files
df_swisseph = pd.concat([pd.DataFrame(multi_eph_data["swe_eph"][ayan]) for ayan in SWE_AYANAMSHAS.keys()])
df_swisseph.reset_index(inplace=True, drop = True)

### Results from `VedicAstro` which uses `flatlib`

In [14]:
flatlib_ayanamshas =  ["Krishnamurti", "Krishnamurti_Senthilathiban", "Lahiri", "Lahiri_1940", "Lahiri_VP285", "Lahiri_ICRC"]
house_system = "Placidus"

flatlib_results = []
for ayan in flatlib_ayanamshas:
    vhd = va.VedicHoroscopeData(year = year, month = month, day = day, hour = hour, minute = minute, second = secs, utc = utc_offset, 
                            latitude = latitude, longitude = longitude, ayanamsa = ayan, house_system = house_system)
    chart = vhd.generate_chart()

    ## Compute results for all houses
    for house in chart.houses:
        _, house_sign, house_dms, _ = str(house).split()
        res = {"Ayanamsa" : ayan, "HouseID": house.id[5:],"Sign": house_sign, "DecLong": house.lon, "DMS" : house_dms}
        res.update(get_rl_nl_sl_data(deg = house.lon))
        flatlib_results.append(res)
        

In [15]:
df_flatlib = pd.DataFrame(flatlib_results)

In [16]:
## Ayanamsa-wise comparison
req_ayanamsa = "Krishnamurti_Senthilathiban"
df1_html = pd.DataFrame(multi_eph_data["swe_eph"][req_ayanamsa]).to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, ayan = req_ayanamsa)

,Ayanamsa,HouseID,Sign,DecLong,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Krishnamurti_Senthilathiban,1,Leo,140.570036,20:34:12,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Krishnamurti_Senthilathiban,2,Virgo,167.696585,17:41:47,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Krishnamurti_Senthilathiban,3,Libra,197.988661,17:59:19,Svati-4,Venus,Rahu,Sun,Ketu
3,Krishnamurti_Senthilathiban,4,Scorpio,229.813822,19:48:49,Jyeshtha-1,Mars,Mercury,Venus,Sun
4,Krishnamurti_Senthilathiban,5,Sagittarius,261.497059,21:29:49,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Krishnamurti_Senthilathiban,6,Capricorn,292.012698,22:00:45,Shravana-4,Saturn,Moon,Venus,Saturn
6,Krishnamurti_Senthilathiban,7,Aquarius,320.570036,20:34:12,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Mercury
7,Krishnamurti_Senthilathiban,8,Pisces,347.696585,17:41:47,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Krishnamurti_Senthilathiban,9,Aries,17.988661,17:59:19,Bharani-2,Mars,Venus,Mars,Venus
9,Krishnamurti_Senthilathiban,10,Taurus,49.813822,19:48:49,Rohini-3,Venus,Moon,Ketu,Venus


In [17]:
## Ayanamsa-wise comparison
req_ayanamsa = "Krishnamurti"
df1_html = pd.DataFrame(multi_eph_data["swe_eph"][req_ayanamsa]).to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, ayan = req_ayanamsa)

,Ayanamsa,HouseID,Sign,DecLong,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Krishnamurti,1,Leo,140.590161,20:35:24,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Krishnamurti,2,Virgo,167.716710,17:43:00,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Krishnamurti,3,Libra,198.008786,18:00:31,Svati-4,Venus,Rahu,Sun,Venus
3,Krishnamurti,4,Scorpio,229.833947,19:50:02,Jyeshtha-1,Mars,Mercury,Venus,Moon
4,Krishnamurti,5,Sagittarius,261.517184,21:31:01,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Krishnamurti,6,Capricorn,292.032823,22:01:58,Shravana-4,Saturn,Moon,Venus,Saturn
6,Krishnamurti,7,Aquarius,320.590161,20:35:24,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Mercury
7,Krishnamurti,8,Pisces,347.716710,17:43:00,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Krishnamurti,9,Aries,18.008786,18:00:31,Bharani-2,Mars,Venus,Mars,Sun
9,Krishnamurti,10,Taurus,49.833947,19:50:02,Rohini-3,Venus,Moon,Ketu,Venus


In [18]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri"
df1_html = pd.DataFrame(multi_eph_data["swe_eph"][req_ayanamsa]).to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, ayan = req_ayanamsa)

,Ayanamsa,HouseID,Sign,DecLong,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Lahiri,1,Leo,140.493308,20:29:35,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Lahiri,2,Virgo,167.619858,17:37:11,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Lahiri,3,Libra,197.911934,17:54:42,Svati-4,Venus,Rahu,Sun,Mercury
3,Lahiri,4,Scorpio,229.737094,19:44:13,Jyeshtha-1,Mars,Mercury,Venus,Sun
4,Lahiri,5,Sagittarius,261.420331,21:25:13,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Lahiri,6,Capricorn,291.935970,21:56:09,Shravana-4,Saturn,Moon,Venus,Saturn
6,Lahiri,7,Aquarius,320.493308,20:29:35,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Saturn
7,Lahiri,8,Pisces,347.619858,17:37:11,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Lahiri,9,Aries,17.911934,17:54:42,Bharani-2,Mars,Venus,Mars,Venus
9,Lahiri,10,Taurus,49.737094,19:44:13,Rohini-3,Venus,Moon,Ketu,Venus


In [19]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri_1940"
df1_html = pd.DataFrame(multi_eph_data["swe_eph"][req_ayanamsa]).to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, ayan = req_ayanamsa)

,Ayanamsa,HouseID,Sign,DecLong,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Lahiri_1940,1,Leo,140.508078,20:30:29,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Lahiri_1940,2,Virgo,167.634627,17:38:04,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Lahiri_1940,3,Libra,197.926703,17:55:36,Svati-4,Venus,Rahu,Sun,Mercury
3,Lahiri_1940,4,Scorpio,229.751863,19:45:06,Jyeshtha-1,Mars,Mercury,Venus,Sun
4,Lahiri_1940,5,Sagittarius,261.435100,21:26:06,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Lahiri_1940,6,Capricorn,291.950740,21:57:02,Shravana-4,Saturn,Moon,Venus,Saturn
6,Lahiri_1940,7,Aquarius,320.508078,20:30:29,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Saturn
7,Lahiri_1940,8,Pisces,347.634627,17:38:04,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Lahiri_1940,9,Aries,17.926703,17:55:36,Bharani-2,Mars,Venus,Mars,Venus
9,Lahiri_1940,10,Taurus,49.751863,19:45:06,Rohini-3,Venus,Moon,Ketu,Venus


In [20]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri_VP285"
df1_html = pd.DataFrame(multi_eph_data["swe_eph"][req_ayanamsa]).to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, ayan = req_ayanamsa)

,Ayanamsa,HouseID,Sign,DecLong,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Lahiri_VP285,1,Leo,140.486920,20:29:12,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Lahiri_VP285,2,Virgo,167.613469,17:36:48,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Lahiri_VP285,3,Libra,197.905545,17:54:19,Svati-4,Venus,Rahu,Sun,Mercury
3,Lahiri_VP285,4,Scorpio,229.730705,19:43:50,Jyeshtha-1,Mars,Mercury,Venus,Sun
4,Lahiri_VP285,5,Sagittarius,261.413942,21:24:50,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Lahiri_VP285,6,Capricorn,291.929582,21:55:46,Shravana-4,Saturn,Moon,Venus,Saturn
6,Lahiri_VP285,7,Aquarius,320.486920,20:29:12,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Saturn
7,Lahiri_VP285,8,Pisces,347.613469,17:36:48,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Lahiri_VP285,9,Aries,17.905545,17:54:19,Bharani-2,Mars,Venus,Mars,Venus
9,Lahiri_VP285,10,Taurus,49.730705,19:43:50,Rohini-3,Venus,Moon,Ketu,Venus


In [21]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri_ICRC"
df1_html = pd.DataFrame(multi_eph_data["swe_eph"][req_ayanamsa]).to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, ayan = req_ayanamsa)

,Ayanamsa,HouseID,Sign,DecLong,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Lahiri_ICRC,1,Leo,140.493612,20:29:37,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Lahiri_ICRC,2,Virgo,167.620161,17:37:12,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Lahiri_ICRC,3,Libra,197.912237,17:54:44,Svati-4,Venus,Rahu,Sun,Mercury
3,Lahiri_ICRC,4,Scorpio,229.737398,19:44:14,Jyeshtha-1,Mars,Mercury,Venus,Sun
4,Lahiri_ICRC,5,Sagittarius,261.420635,21:25:14,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Lahiri_ICRC,6,Capricorn,291.936274,21:56:10,Shravana-4,Saturn,Moon,Venus,Saturn
6,Lahiri_ICRC,7,Aquarius,320.493612,20:29:37,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Saturn
7,Lahiri_ICRC,8,Pisces,347.620161,17:37:12,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Lahiri_ICRC,9,Aries,17.912237,17:54:44,Bharani-2,Mars,Venus,Mars,Venus
9,Lahiri_ICRC,10,Taurus,49.737398,19:44:14,Rohini-3,Venus,Moon,Ketu,Venus


In [22]:
# Convert DataFrames to HTML and concatenate them with some space in between
df1_html = df_swisseph.to_html()
df2_html = df_flatlib.to_html()
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, ayan = "For All")

,Ayanamsa,HouseID,Sign,DecLong,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Krishnamurti,1,Leo,140.590161,20:35:24,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Krishnamurti,2,Virgo,167.716710,17:43:00,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Krishnamurti,3,Libra,198.008786,18:00:31,Svati-4,Venus,Rahu,Sun,Venus
3,Krishnamurti,4,Scorpio,229.833947,19:50:02,Jyeshtha-1,Mars,Mercury,Venus,Moon
4,Krishnamurti,5,Sagittarius,261.517184,21:31:01,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Krishnamurti,6,Capricorn,292.032823,22:01:58,Shravana-4,Saturn,Moon,Venus,Saturn
6,Krishnamurti,7,Aquarius,320.590161,20:35:24,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Mercury
7,Krishnamurti,8,Pisces,347.716710,17:43:00,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Krishnamurti,9,Aries,18.008786,18:00:31,Bharani-2,Mars,Venus,Mars,Sun
9,Krishnamurti,10,Taurus,49.833947,19:50:02,Rohini-3,Venus,Moon,Ketu,Venus
